# Analyse von Nachrichtenartikeln in deutscher Sprache
In diesem Notebook verwenden wir verschiedene NLP-Tools von Hugging Face, um große Datensätze von Nachrichtenartikeln in deutscher Sprache zu analysieren. Mit Hilfe von Transformer-Modellen führen wir Sentiment-Analyse, Named Entity Recognition (NER) und Topic Modeling durch, um wichtige Einblicke in die Artikel zu gewinnen.

### Ziel:
- Laden und Verarbeiten großer JSON-Datensätze in Chunks
- Durchführung von Sentiment-Analyse und NER auf Artikeldaten
- Visualisierung und Analyse der Ergebnisse

**Benötigte Zeit:** ~2 Stunden


## 1. Setup und Laden der JSON-Daten in Chunks
Wir beginnen mit der Installation der notwendigen Bibliotheken und laden die JSON-Daten in verwaltbaren Chunks, damit wir große Dateien effizient verarbeiten können.

In [ ]:
!pip install pandas transformers

In [ ]:

# Bibliotheken importieren
import json
import pandas as pd
from transformers import pipeline

# JSON-Daten in Chunks laden
def load_json_from_github_in_chunks(url, chunk_size=1000):
    response = requests.get(url)
    data = response.json()
    articles = list(data.values())
    for i in range(0, len(articles), chunk_size):
        yield articles[i:i + chunk_size]

# Pfad zur Datei angeben (Pfad entsprechend in Google Colab anpassen)
github_url = 'https://raw.githubusercontent.com/dajo51/hands-on-msc-journalismus/main/data_2009.json'

# Beispiel-Chunks laden und Struktur inspizieren
sample_chunk = next(load_json_from_github_in_chunks(github_url))
df_sample = pd.DataFrame(sample_chunk)
df_sample.head()
    

## 2. Extrahieren und Verarbeiten der Schlüsselinformationen
Wir transformieren die JSON-Daten in ein DataFrame-Format, um sie leichter analysieren zu können. Hier extrahieren wir Felder wie `title`, `description`, `text` und `keywords`.

In [ ]:

# Funktion zum Extrahieren der Schlüssel-Felder
def extract_fields(article_chunk):
    data = []
    for article in article_chunk:
        data.append({
            "title": article.get("title"),
            "description": article.get("description"),
            "text": article.get("text"),
            "keywords": article.get("keywords"),
            "author": article.get("author"),
            "date": article.get("date")
        })
    return pd.DataFrame(data)

# Beispiel-Chunks verarbeiten
df_articles = extract_fields(sample_chunk)
df_articles.head()
    

## 3. Sentiment-Analyse auf Artikelbeschreibungen
Wir verwenden ein Transformer-Modell, das speziell für die deutsche Sprache trainiert wurde, um die Stimmung der Artikelbeschreibungen zu analysieren.

In [ ]:

# Sentiment-Analyse Pipeline mit einem deutschen Modell initialisieren
sentiment_analyzer = pipeline("sentiment-analysis", model="oliverguhr/german-sentiment-bert")

# Sentiment-Analyse auf die Beschreibungen anwenden
df_articles['sentiment'] = df_articles['description'].apply(lambda x: sentiment_analyzer(x)[0]['label'] if pd.notnull(x) else None)

# Ergebnisse anzeigen
df_articles[['title', 'description', 'sentiment']].head()
    

## 4. Named Entity Recognition (NER) auf Artikeltexten
Mit Hilfe eines multilingualen NER-Modells analysieren wir die wichtigsten Entitäten (z. B. Personen, Organisationen, Orte), die in den Artikeln genannt werden.

In [ ]:

# NER-Pipeline mit einem multilingualen Modell für die deutsche Sprache
ner_analyzer = pipeline("ner", model="Davlan/xlm-roberta-base-ner-hrl", grouped_entities=True)

# NER auf den Artikeltext anwenden
df_articles['entities'] = df_articles['text'].apply(lambda x: ner_analyzer(x) if pd.notnull(x) else None)

# Ergebnisse anzeigen
df_articles[['title', 'entities']].head()
    

## 5. Keyword-Extraktion aus dem Keyword-Feld
Wir analysieren die häufig vorkommenden Schlagwörter, um wiederkehrende Themen zu identifizieren.

In [ ]:

from collections import Counter

# Schlagwörter splitten und Häufigkeiten zählen
all_keywords = df_articles['keywords'].dropna().str.split(', ').sum()
keyword_counts = Counter(all_keywords)

# Umwandeln in DataFrame zur Visualisierung
df_keywords = pd.DataFrame(keyword_counts.items(), columns=['Keyword', 'Frequency']).sort_values(by='Frequency', ascending=False)
df_keywords.head(10)
    

## 6. Topic Modeling auf Artikeltexten
Mit Topic Modeling können wir zugrunde liegende Themen in den Artikeln identifizieren.

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Textdaten für Topic Modeling vorbereiten
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='german')
text_matrix = vectorizer.fit_transform(df_articles['text'].dropna())

# LDA-Analyse durchführen
lda = LatentDirichletAllocation(n_components=5, random_state=0)
lda.fit(text_matrix)

# Top-Wörter für jedes Thema anzeigen
for index, topic in enumerate(lda.components_):
    print(f"Thema #{index+1}:")
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print("\n")
    

## 7. Visualisierung der Sentiment- und Keyword-Analyse
Abschließend visualisieren wir die Häufigkeit der wichtigsten Schlagwörter sowie die Verteilung der Stimmung in den Artikelbeschreibungen.

In [ ]:

import matplotlib.pyplot as plt

# Top-Schlagwörter plotten
plt.figure(figsize=(10, 6))
df_keywords.head(10).plot(kind='barh', x='Keyword', y='Frequency', legend=False, color='skyblue')
plt.title('Top-Schlagwörter in Artikeln')
plt.xlabel('Häufigkeit')
plt.show()

# Sentiment-Verteilung plotten
sentiment_counts = df_articles['sentiment'].value_counts()
plt.figure(figsize=(6, 6))
sentiment_counts.plot(kind='pie', autopct='%1.1f%%', startangle=140)
plt.title('Sentiment-Verteilung in Artikelbeschreibungen')
plt.show()
    

## Zusammenfassung
In diesem Notebook haben wir verschiedene NLP-Methoden auf einen großen Datensatz von deutschen Nachrichtenartikeln angewendet. Durch Sentiment-Analyse, Named Entity Recognition und Topic Modeling haben wir nützliche Erkenntnisse aus den Artikeldaten gewonnen. Diese Methoden können weiter genutzt werden, um tiefere Analysen in journalistischen Projekten durchzuführen.

**Vielen Dank für die Teilnahme!**